# Pandas tutorial

In [ ]:
import pandas as pd

pd.DataFrame({'Yes': [50, 21], 'No': [131, 2]})

pd.DataFrame({'Bob': ['I liked it.', 'It was awful.'], 'Sue': ['Pretty good.', 'Bland.']})

pd.DataFrame({'Bob': ['I liked it.', 'It was awful.'], 
              'Sue': ['Pretty good.', 'Bland.']},
             index=['Product A', 'Product B'])

In [ ]:
pd.Series([1, 2, 3, 4, 5])

pd.Series([30, 35, 40], index=['2015 Sales', '2016 Sales', '2017 Sales'], name='Product A')

In [ ]:
wine_reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv")

wine_reviews.shape

In [ ]:
import pandas as pd
reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)
pd.set_option('max_rows', 5)

In [ ]:
reviews.iloc[0] # pandas diff native python : row first, column second

reviews.iloc[:, 0]

reviews.iloc[:3, 0]

reviews.iloc[[0, 1, 2], 0]

reviews.iloc[-5:]

reviews.loc[0, 'country'] #label based

reviews.loc[:, ['taster_name', 'taster_twitter_handle', 'points']]

## Choosing between loc and iloc

When choosing or transitioning between loc and iloc, there is one "gotcha" worth keeping in mind, which is that the two methods use slightly different indexing schemes.

iloc uses the Python stdlib indexing scheme, where the first element of the range is included and the last one excluded. So 0:10 will select entries 0,...,9. loc, meanwhile, indexes inclusively. So 0:10 will select entries 0,...,10.

Why the change? Remember that loc can index any stdlib type: strings, for example. If we have a DataFrame with index values Apples, ..., Potatoes, ..., and we want to select "all the alphabetical fruit choices between Apples and Potatoes", then it's a lot more convenient to index df.loc['Apples':'Potatoes'] than it is to index something like df.loc['Apples', 'Potatoet] (t coming after s in the alphabet).

In [ ]:
reviews.set_index("title")

In [ ]:
reviews.country == 'Italy'

In [ ]:
reviews.loc[reviews.country == 'Italy']

reviews.loc[(reviews.country == 'Italy') & (reviews.points >= 90)]

reviews.loc[(reviews.country == 'Italy') | (reviews.points >= 90)]

reviews.loc[reviews.country.isin(['Italy', 'France'])]

reviews.loc[reviews.price.notnull()] #isnull()

In [ ]:
reviews['index_backwards'] = range(len(reviews), 0, -1)

In [ ]:
reviews.points.describe() #it is type aware, depending on the feature !

reviews.taster_name.describe()

reviews.points.mean()

reviews.taster_name.unique()

reviews.taster_name.value_counts()

In [ ]:
review_points_mean = reviews.points.mean()
reviews.points.map(lambda p: p - review_points_mean)

def remean_points(row):
    row.points = row.points - review_points_mean
    return row

reviews.apply(remean_points, axis='columns')

In [ ]:
bargain_idx = (reviews.points / reviews.price).idxmax()
bargain_wine = reviews.loc[bargain_idx, 'title']

In [ ]:
n_trop = reviews.description.map(lambda desc: "tropical" in desc).sum()
n_fruity = reviews.description.map(lambda desc: "fruity" in desc).sum()
descriptor_counts = pd.Series([n_trop, n_fruity], index=['tropical', 'fruity'])

In [ ]:
def stars(row):
    if row.country == 'Canada':
        return 3
    elif row.points >= 95:
        return 3
    elif row.points >= 85:
        return 2
    else:
        return 1

star_ratings = reviews.apply(stars, axis='columns')

## group by and sorting

In [ ]:
import pandas as pd
reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)
pd.set_option("display.max_rows", 5)

In [ ]:
reviews.groupby('points').points.count() #longer version of value_counts

reviews.groupby('points').price.min() #

reviews_written = reviews.groupby('taster_twitter_handle').size()

reviews.groupby('winery').apply(lambda df: df.title.iloc[0])

reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()]) #create multi index !! see next line

countries_reviewed.reset_index()

In [ ]:
reviews.groupby(['country']).price.agg([len, min, max])

In [ ]:
countries_reviewed = countries_reviewed.reset_index()

countries_reviewed.sort_values(by='len')
countries_reviewed.sort_values(by='len', ascending=False)
countries_reviewed.sort_index()
countries_reviewed.sort_values(by=['country', 'len'])

In [ ]:
best_rating_per_price = reviews.groupby('price')['points'].max().sort_index()

price_extremes = reviews.groupby('variety').price.agg([min, max])

sorted_varieties = price_extremes.sort_values(by=['min', 'max'], ascending=False)

reviewer_mean_ratings = reviews.groupby('taster_name').points.mean()

country_variety_counts = reviews.groupby(['country', 'variety']).size().sort_values(ascending=False)

The data type for a column in a DataFrame or a Series is known as the dtype.

In [ ]:
reviews.dtypes

reviews.price.dtype

reviews.points.astype('float64')

reviews[pd.isnull(reviews.country)] #notnull()

reviews.region_2.fillna("Unknown")# can fill with other things

reviews.taster_twitter_handle.replace("@kerinokeefe", "@kerino")

In [ ]:
point_strings = reviews.points.astype(str)

In [ ]:
reviews.rename(columns={'points': 'score'})

reviews.rename(index={0: 'firstEntry', 1: 'secondEntry'})

reviews.rename_axis("wines", axis='rows').rename_axis("fields", axis='columns') # change name of axes

In [ ]:
canadian_youtube = pd.read_csv("../input/youtube-new/CAvideos.csv")
british_youtube = pd.read_csv("../input/youtube-new/GBvideos.csv")

pd.concat([canadian_youtube, british_youtube]) # only if same type of dataset, same columns rows

In [ ]:
left = canadian_youtube.set_index(['title', 'trending_date'])
right = british_youtube.set_index(['title', 'trending_date'])

left.join(right, lsuffix='_CAN', rsuffix='_UK') # if columns names are different works 

In [ ]:
renamed = reviews.rename(columns=dict(region_1='region', region_2='locale'))

powerlifting_combined = powerlifting_meets.set_index("MeetID").join(powerlifting_competitors.set_index("MeetID"))